In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Declaring Continuous Variables

## Calling add_input and add_output
Every component in an OpenMDAO model is an instance of `ExplicitComponent`, `ImplicitComponent`, or a subclass of one of these classes. Regardless of the type, each component has input variables and output variables that it must declare.

In explicit and implicit components, the user must call `add_input` and `add_output` to declare variables in the `setup` method. An example is given below.

In [2]:
import openmdao.api as om


class TestExplCompSimple(om.ExplicitComponent):

    def setup(self):
        self.add_input('length', val=1., desc='length of rectangle')
        self.add_input('width', val=1., desc='width of rectangle')
        self.add_output('area', val=1., desc='area of rectangle')

    def setup_partials(self):
        self.declare_partials('*', '*')

    def compute(self, inputs, outputs):
        outputs['area'] = inputs['length'] * inputs['width']


In [3]:
"""Just to run the code for TestExplCompSimple."""
from openmdao.utils.assert_utils import assert_near_equal

p = om.Problem()
p.model.add_subsystem('comp', TestExplCompSimple(), promotes=['*'])
p.setup()

assert_near_equal(p.get_val('length'), 1.)

0.0

```{note}
Variable names have few restrictions, but the following characters are not allowed in a variable name: ‘.’, ‘*’, ‘?’, ‘!’, ‘[‘, ‘]’.
```


## Method Signatures

```{eval-rst}
    .. automethod:: openmdao.core.component.Component.add_input
        :noindex:
```
```{eval-rst}
    .. automethod:: openmdao.core.component.Component.add_output
        :noindex:
```

## Usage

1\. Declaring with only the default value. 


In [4]:
import numpy as np

class CompAddWithDefault(om.ExplicitComponent):
    """Component for tests for declaring only default value."""

    def setup(self):
        self.add_input('x_a')
        self.add_input('x_b', val=3.)
        self.add_input('x_c', val=(3., 3.))
        self.add_input('x_d', val=[3., 3.])
        self.add_input('x_e', val=3. * np.ones((2, 2)))

        self.add_output('y_a')
        self.add_output('y_b', val=6.)
        self.add_output('y_c', val=(6., 6., 6.))
        self.add_output('y_d', val=[6., 6., 6.])
        self.add_output('y_e', val=6. * np.ones((3, 2)))

In [5]:
"""Test declaring only default value."""
p = om.Problem()
p.model.add_subsystem('comp', CompAddWithDefault(), promotes=['*'])
p.setup()

print(p.get_val('x_a'))

[1.]


In [6]:
print(p.get_val('x_b'))

[3.]


In [7]:
print(p.get_val('x_c'))

[3. 3.]


In [8]:
print(p.get_val('x_d'))

[3. 3.]


In [9]:
print(p.get_val('x_e'))

[[3. 3.]
 [3. 3.]]


In [10]:
print(p.get_val('y_a'))

[1.]


In [11]:
print(p.get_val('y_b'))

[6.]


In [12]:
print(p.get_val('y_c'))

[6. 6. 6.]


In [13]:
print(p.get_val('y_d'))

[6. 6. 6.]


In [14]:
print(p.get_val('y_e'))

[[6. 6.]
 [6. 6.]
 [6. 6.]]


In [15]:
assert_near_equal(p.get_val('x_a'), 1.)
assert_near_equal(p.get_val('x_b'), 3.)
assert_near_equal(p.get_val('x_c'), 3. * np.ones(2))
assert_near_equal(p.get_val('x_d'), 3. * np.ones(2))
assert_near_equal(p.get_val('x_e'), 3. * np.ones((2, 2)))
assert_near_equal(p.get_val('y_a'), 1.)
assert_near_equal(p.get_val('y_b'), 6.)
assert_near_equal(p.get_val('y_c'), 6. * np.ones(3))
assert_near_equal(p.get_val('y_d'), 6. * np.ones(3))
assert_near_equal(p.get_val('y_e'), 6. * np.ones((3, 2)))

0.0

2\. Declaring with only the *shape* argument.


In [16]:
class CompAddWithShape(om.ExplicitComponent):
    """Component for tests for declaring only shape."""

    def setup(self):
        self.add_input('x_a', shape=2)
        self.add_input('x_b', shape=(2, 2))
        self.add_input('x_c', shape=[2, 2])

        self.add_output('y_a', shape=3)
        self.add_output('y_b', shape=(3, 3))
        self.add_output('y_c', shape=[3, 3])

In [17]:
"""Test declaring only shape."""
p = om.Problem()
p.model.add_subsystem('comp', CompAddWithShape(), promotes=['*'])
p.setup()

print(p.get_val('x_a'))

[1. 1.]


In [18]:
print(p.get_val('x_b'))

[[1. 1.]
 [1. 1.]]


In [19]:
print(p.get_val('x_c'))

[[1. 1.]
 [1. 1.]]


In [20]:
print(p.get_val('y_a'))

[1. 1. 1.]


In [21]:
print(p.get_val('y_b'))

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [22]:
print(p.get_val('y_c'))

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [23]:
assert_near_equal(p.get_val('x_a'), np.ones(2))
assert_near_equal(p.get_val('x_b'), np.ones((2, 2)))
assert_near_equal(p.get_val('x_c'), np.ones((2, 2)))
assert_near_equal(p.get_val('y_a'), np.ones(3))
assert_near_equal(p.get_val('y_b'), np.ones((3, 3)))
assert_near_equal(p.get_val('y_c'), np.ones((3, 3)))

0.0


4\. Declaring an array variable with a scalar default value.


In [24]:
class CompAddArrayWithScalar(om.ExplicitComponent):
    """Component for tests for declaring a scalar val with an array variable."""

    def setup(self):
        self.add_input('x_a', val=2.0, shape=(6))
        self.add_input('x_b', val=2.0, shape=(3, 2))
        self.add_output('y_a', val=3.0, shape=(6))
        self.add_output('y_b', val=3.0, shape=(3, 2))

In [25]:
"""Test declaring a scalar val with an array variable."""
p = om.Problem()
p.model.add_subsystem('comp', CompAddArrayWithScalar(), promotes=['*'])
p.setup()

In [26]:
print(p.get_val('x_a'))

[2. 2. 2. 2. 2. 2.]


In [27]:
print(p.get_val('x_b'))

[[2. 2.]
 [2. 2.]
 [2. 2.]]


In [28]:
print(p.get_val('y_a'))

[3. 3. 3. 3. 3. 3.]


In [29]:
print(p.get_val('y_b'))

[[3. 3.]
 [3. 3.]
 [3. 3.]]


In [30]:
assert_near_equal(p.get_val('x_a'), 2. * np.ones(6))
assert_near_equal(p.get_val('x_b'), 2. * np.ones((3, 2)))
assert_near_equal(p.get_val('y_a'), 3. * np.ones(6))
assert_near_equal(p.get_val('y_b'), 3. * np.ones((3, 2)))

0.0

6\. Declaring an output with bounds, using *upper* and/or *lower* arguments.


In [31]:
class CompAddWithBounds(om.ExplicitComponent):
    """Component for tests for declaring bounds."""

    def setup(self):
        self.add_input('x')

        self.add_output('y_a', val=2.0, lower=0.)
        self.add_output('y_b', val=2.0, lower=0., upper=10.)
        self.add_output('y_c', val=2.0 * np.ones(6),  lower=np.zeros(6), upper=10.)
        self.add_output('y_d', val=2.0 * np.ones(6), lower=0., upper=[12, 10, 10, 10, 10, 12])
        self.add_output('y_e', val=2.0 * np.ones((3, 2)), lower=np.zeros((3, 2)))

In [32]:
"""Test declaring bounds."""
p = om.Problem()
p.model.add_subsystem('comp', CompAddWithBounds(), promotes=['*'])
p.setup()

print(p.get_val('y_a'))

[2.]


In [33]:
print(p.get_val('y_b'))

[2.]


In [34]:
print(p.get_val('y_c'))

[2. 2. 2. 2. 2. 2.]


In [35]:
print(p.get_val('y_d'))

[2. 2. 2. 2. 2. 2.]


In [36]:
print(p.get_val('y_e'))

[[2. 2.]
 [2. 2.]
 [2. 2.]]


In [37]:
assert_near_equal(p.get_val('y_a'), 2.)
assert_near_equal(p.get_val('y_b'), 2.)
assert_near_equal(p.get_val('y_c'), 2. * np.ones(6))
assert_near_equal(p.get_val('y_d'), 2. * np.ones(6))
assert_near_equal(p.get_val('y_e'), 2. * np.ones((3, 2)))

0.0

7\. Adding tags to input and output variables. These tags can then be used to filter what gets returned from the `System.get_io_metadata` method and displayed/returned from the `System.list_inputs` and `System.list_outputs` methods.



In [38]:
class RectangleCompWithTags(om.ExplicitComponent):
    """
    A simple Explicit Component that also has input and output with tags.
    """

    def setup(self):
        self.add_input('length', val=1., tags=["tag1", "tag2"])
        self.add_input('width', val=1., tags=["tag2"])
        self.add_output('area', val=1., tags="tag1")

    def setup_partials(self):
        self.declare_partials('*', '*')

    def compute(self, inputs, outputs):
        outputs['area'] = inputs['length'] * inputs['width']

prob = om.Problem()
prob.model.add_subsystem('comp', RectangleCompWithTags(), promotes=['*'])
prob.setup(check=False)
prob.run_model()

# Inputs no tags
inputs = prob.model.list_inputs(val=False, out_stream=None)
print(sorted(inputs))

[('comp.length', {'prom_name': 'length'}), ('comp.width', {'prom_name': 'width'})]


In [39]:
# Inputs with tags
inputs = prob.model.list_inputs(val=False, out_stream=None, tags="tag1")
print(sorted(inputs))

[('comp.length', {'prom_name': 'length'})]


In [40]:
# Inputs with multiple tags
inputs = prob.model.list_inputs(val=False, out_stream=None, tags=["tag1", "tag2"])
print(sorted(inputs))

[('comp.length', {'prom_name': 'length'}), ('comp.width', {'prom_name': 'width'})]


In [41]:
# Inputs with tag that does not match
inputs = prob.model.list_inputs(val=False, out_stream=None, tags="tag3")
print(sorted(inputs))

[]


In [42]:
# Outputs no tags
outputs = prob.model.list_outputs(val=False, out_stream=None)
print(sorted(outputs))

[('comp.area', {'prom_name': 'area'})]


In [43]:
# Outputs with tags
outputs = prob.model.list_outputs(val=False, out_stream=None, tags="tag1")
print(sorted(outputs))

[('comp.area', {'prom_name': 'area'})]


In [44]:
# Outputs with multiple tags
outputs = prob.model.list_outputs(val=False, out_stream=None, tags=["tag1", "tag3"])
print(sorted(outputs))

[('comp.area', {'prom_name': 'area'})]


In [45]:
# Outputs with tag that does not match
outputs = prob.model.list_outputs(val=False, out_stream=None, tags="tag3")
print(sorted(outputs))

[]


In [46]:
# Inputs no tags
inputs = prob.model.list_inputs(val=False, out_stream=None)
assert(sorted(inputs) == [
    ('comp.length', {'prom_name': 'length'}),
    ('comp.width', {'prom_name': 'width'})
])

# Inputs with tags
inputs = prob.model.list_inputs(val=False, out_stream=None, tags="tag1")
assert(sorted(inputs) == [
    ('comp.length', {'prom_name': 'length'})
])

# Inputs with multiple tags
inputs = prob.model.list_inputs(val=False, out_stream=None, tags=["tag1", "tag2"])
assert(sorted(inputs) == [
    ('comp.length', {'prom_name': 'length'}),
    ('comp.width', {'prom_name': 'width'})
])

# Inputs with tag that does not match
inputs = prob.model.list_inputs(val=False, out_stream=None, tags="tag3")
assert(sorted(inputs) == [])

# Outputs no tags
outputs = prob.model.list_outputs(val=False, out_stream=None)
assert(sorted(outputs) == [
    ('comp.area', {'prom_name': 'area'})
])

# Outputs with tags
outputs = prob.model.list_outputs(val=False, out_stream=None, tags="tag1")
assert(sorted(outputs) == [
    ('comp.area', {'prom_name': 'area'})
])

# Outputs with multiple tags
outputs = prob.model.list_outputs(val=False, out_stream=None, tags=["tag1", "tag3"])
assert(sorted(outputs) == [
    ('comp.area', {'prom_name': 'area'})
])

# Outputs with tag that does not match
outputs = prob.model.list_outputs(val=False, out_stream=None, tags="tag3")
assert(sorted(outputs) == [])


## Calling add_residual

By default, residuals and outputs have the same names, but there are times when writing an ImplicitComponent that this might be confusing. [OpenMDAO POEM 69](https://github.com/OpenMDAO/POEMs/blob/master/POEM_069.md) has specific examples that demonstrate the confusion.

To address this, the `add_residual` method was added to ImplicitComponent.

```{eval-rst}
    .. automethod:: openmdao.core.implicitcomponent.ImplicitComponent.add_residual
        :noindex:
```

Note that if you make any calls to `add_residual`, you must use it to add *all* of the residuals, so that the size of the residuals vector is the same size as the outputs vector for that component.

Also note that this residual renaming is only active inside of the component, and the rest of the framework will still deal with residuals as if they map 1-to-1 with outputs.